In [ ]:
source_range.Copy()

# Get the copied data from the clipboard
import win32clipboard
# Open clipboard
win32clipboard.OpenClipboard()
copied_data = win32clipboard.GetClipboardData(win32clipboard.CF_UNICODETEXT)
win32clipboard.CloseClipboard()

In [30]:
# Cost Consistency
# importing modules
import shutil  # for file manipulation
import os
import win32com.client as win32
from ok_sap_script import *
import time
from datetime import date, timedelta, datetime, timezone
import pyautogui
import win32clipboard # work with the clipboard

class Control():
    def __init__(self):
        # main variables
        self.curr_user = os.getlogin()
        self.date_stamp = datetime.today().strftime("%d-%m-%Y")
        self.receiver = "oleksandr.komarov@zoetis.com"
        
        # SAP variables global
        self.environment = "ECC Production"
        
        # SAP variables ZM2D_28
        self.t_code_sap = "ZM2D_28"
        self.variant_sap = "COST_CONS_AUTO"
        self.layout_sap = "CC_AUTO"
        self.period = date.today().strftime("%#m") # use # to remove a leading 0
        self.period_0 = date.today().strftime("%m")
        self.year = datetime.today().year # get current year
            # dates
        self.last_month_last_day = (datetime.today().replace(day=1) - timedelta(days=1)).strftime("%d.%m.%Y")
        self.last_month_first_day = (datetime.today().replace(day=1) - timedelta(days=1)).replace(day=1).strftime("%d.%m.%Y")
        self.last_month =  (datetime.today().replace(day=1) - timedelta(days=1)).strftime("%#m")

        # SAP variables mb51
        self.t_code_sap_mb51 = "MB51"
        self.variant_sap_mb51 = "COST_CONS_AUTO"
        self.layout_sap_mb51 = "/DPM"

        # extract paths zm2d_28
        self.extract_path_zm2d_28 = fr"C:\\Users\\{self.curr_user}\\Desktop\\Automations\\CPT_TP\\Cost_consistency\\Output\\"
        self.extract_name_zm2d_28 = f"zm2d_28_extract_for_{self.date_stamp}.xlsx"

        # extract paths mb51
        self.extract_path_mb51 = fr"C:\\Users\\{self.curr_user}\\Desktop\\Automations\\CPT_TP\\Cost_consistency\\Output\\"
        self.extract_name_mb51 = f"mb51_extract_for_{self.date_stamp}.xlsx"

        # Paths to files
            # model
        self.model_file_path = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Cost_consistency/Cost_consistency_model.xlsx"
        self.new_file_path = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Cost_consistency/Output/Cost_consistency_for_{self.date_stamp}.xlsx"
            # zm2d_28
        self.zm2d_28_file_path = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Cost_consistency/Output/{self.extract_name_zm2d_28}"
            #mb51
        self.mb51_file_path = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Cost_consistency/Output/{self.extract_name_mb51}"
            # screenshot
        self.screenshot_path = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Cost_consistency/Screenshots/"

            # historical costs
        self.final_path_to_historical_cost = f"S:/Robotics_COE_Prod/RPA/MM60/Outputs/{self.year}/{self.period_0}/"
        self.historical_cost_file = f"Historical cost AP{self.period_0} LE2941.xlsm"
        self.final_joined_path_to_historical_cost = f"{self.final_path_to_historical_cost}{self.historical_cost_file}"
            # same but on my local drive
        self.new_file_path_to_historical_cost = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Cost_consistency/Workings/{self.historical_cost_file}"

    def __call__(self, *args, **kwargs):
        self.run_logic_sap_zm2d_28(self.t_code_sap) # extract zm2d_28 from SAP
        self.run_logic_excel() # paste hictorical cost and zm2d_28 in Excel, add screenshots
        self.run_logic_excel_part_2() # final part of working with mb51 excel and main file 
        close_excel()

    def sap_decorator(sap_function):
        '''Closing SAP sessions before and after running a script'''
        def sap_wrapper(self, t_code_sap, *args, **kwargs):
            sap_close() # ensure there are no open session
            open_sap = sap_open() # open new session
            time.sleep(5) # make sure SAP opens up
            if open_sap == True:
                self.session = sap_logon(environment=self.environment, client=1)
                sap_code(tcode=t_code_sap, session=self.session)
                sap_function(self) # this is the main fuction to be decorated
            else:
                print("SAP did not open")
            sap_close() # close SAP
        return sap_wrapper

    def excel_decorator(excel_function):
        '''Opening Excel, applying some standard parameters for runtime optimisation, etc.'''
        def excel_wrapper(self, *args, **kwargs):
            try:
                # default Excel runtime optimisation
                self.excel = win32.Dispatch("Excel.Application")
                self.excel.AskToUpdateLinks = False
                self.excel.DisplayAlerts = False
                self.excel.Visible = True
                self.excel.ScreenUpdating = False
                excel_function(self) # our main Excel function to run
            except Exception as e:
                # return to a normal Excel, then close it
                self.excel.ScreenUpdating = True
                self.excel.Application.Calculation = -4105  # to set xlCalculationAutomatic
                close_excel()
                print(f"something went wrong, namely\n{e}")
        return excel_wrapper
    
    def send_email(self):
        # send by email
            outlook = win32.Dispatch('outlook.application')
            mail = outlook.CreateItem(0)
            mail.To = self.receiver
            mail.Subject = f"Automatic email for price Cost consistency control"
            mail.Body = f"Please consult the attachment"
            mail.Attachments.Add(f"{self.new_file_path}")
            #mail.Display(True)
            mail.Send()

    @sap_decorator
    def run_logic_sap_zm2d_28(self):
        # self.session is defined inside sap_decorator
            try:
                sap_variant(session=self.session, var_to_use=self.variant_sap)
            except: 
                sap_variant(session=self.session, var_to_use=self.variant_sap, version=2) # in our case, V2 is the most probable scenario
            # adjust the dates
            self.session.findById("wnd[0]/usr/txtSP$00001-LOW").text = self.last_month
            self.session.findById("wnd[0]/usr/txtSP$00002-LOW").text = self.year
            self.session.findById("wnd[0]/usr/ctxt%LAYOUT").text = self.layout_sap
            sap_run(session=self.session) # run report
             # take screenshots of first and last pages
            #pyautogui.hotkey('ctrl', 'end') # I almist feel this one is cheating
            self.session.findById("wnd[0]/usr/cntlCONTAINER/shellcont/shell").pressToolbarContextButton("&MB_VIEW")
            self.session.findById("wnd[0]/usr/cntlCONTAINER/shellcont/shell").selectContextMenuItem("&PRINT_BACK_PREVIEW")
            self.session.findById("wnd[0]/usr/lbl[12,6]").setFocus() # just in case
            try: # first page
                screenshot_first_page_zm2d_28 = pyautogui.screenshot()
                screenshot_first_page_zm2d_28.save(fr'C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Cost_consistency/Screenshots/scr1_for_{self.extract_name_zm2d_28[:-5]}.png')
            except Exception as e: print(e)
            sap_screen_nagivation(session=self.session, action="down")
            try: # last page
                time.sleep(3)
                screenshot_second_page_zm2d_28 = pyautogui.screenshot()
                screenshot_second_page_zm2d_28.save(fr'C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Cost_consistency/Screenshots/scr2_for_{self.extract_name_zm2d_28[:-5]}.png')
            except Exception as e: print(e)
            sap_extract(session=self.session, extr_path=self.extract_path_zm2d_28, extr_name=self.extract_name_zm2d_28)
        # return to default
            close_excel() # close excel
   
    @excel_decorator
    def run_logic_excel(self):
        # copy model file
        try:
            shutil.copy(self.model_file_path, self.new_file_path)
        except PermissionError as e:
            print("The file may be opened")
            os.system("taskkill /f /im  excel.exe")
    
    # working with historical cost file
        # copy historical cost file
        try:
            shutil.copy(self.final_joined_path_to_historical_cost, self.new_file_path_to_historical_cost)
        except PermissionError as e:
            print("Historical file is either unreachable, missing or opened")
            os.system("taskkill /f /im  excel.exe")
        # open created Excel file and perform the following manipulations
            # open new file and disable calculations
        self.new_file = self.excel.Workbooks.Open(self.new_file_path) # main file
        self.new_file_hc = self.excel.Workbooks.Open(self.new_file_path_to_historical_cost)
        self.excel.Application.Calculation = (-4135)  # to set xlCalculationManual # Workbook needs to be opened
            # other steps 
        self.new_file_hc_ws = self.new_file_hc.Sheets("MM60 Report") # open historical cost file
        self.last_row_historical_file_ws = self.new_file_hc_ws.Cells(1, 1).End(-4121).Row -1 # find last row of this file # do not include totals
        print(self.last_row_historical_file_ws)

        # open main file and paste the values from mm60 to hc sheet
        self.new_file_hcost_ws = self.new_file.Worksheets("HC")
        self.new_file_hc_ws.Range(f"A2:G{self.last_row_historical_file_ws}").SpecialCells(12).Copy(Destination=self.new_file_hcost_ws.Range(f"B2")) 
        # add concatenate formula =IF(ISBLANK(C2),"",B2&"-"&C2)
        self.new_file_hcost_ws.Range(f"A2:A{self.last_row_historical_file_ws}").Formula = f'=IF(ISBLANK(C2),"",B2&"-"&C2)'
        self.new_file_hc.Close() # close RPA file with hc 
        self.new_file.Save() # save the main file
    # with that we can forget hc file and never think of it anymore
    # working with zm2d_28 extract from SAP
        try:
            # work with the main file
            self.new_file_zm2d_28_ws = self.new_file.Worksheets("ZM2D_28") # navigate to the respective ws
            self.last_row_new_file_zm2d_28_ws = self.new_file_zm2d_28_ws.Cells(1, 1).End(-4121).Row # last row of the current data set
            self.new_file_zm2d_28_ws.Range(f"A2:AE{self.last_row_new_file_zm2d_28_ws}").ClearContents() # prepare the space
            # work with zm2d_28 extract
            self.zm2d_28_file = self.excel.Workbooks.Open(self.zm2d_28_file_path) # open zm2d_28 sap extract
            self.zm2d_28_ws = self.zm2d_28_file.Sheets(1) # open first and only ws it has
            self.last_row_zm2d_28_ws = self.zm2d_28_ws.Cells(1, 1).End(-4121).Row -1 # we do not need to include the totals
            self.zm2d_28_ws.Range(f"A2:AA{self.last_row_zm2d_28_ws}").SpecialCells(12).Copy(Destination=self.new_file_zm2d_28_ws.Range(f"D2")) # copy info
            # add formulas
            self.new_file_zm2d_28_ws.Range(f"A2:A{self.last_row_zm2d_28_ws}").Formula = f'=IF(D2="","",E2&"-"&J2)'
            self.new_file_zm2d_28_ws.Range(f"B2:B{self.last_row_zm2d_28_ws}").Formula = f'=IF(D2="","",IF(W2=AA2,"NO","YES"))'
            self.new_file_zm2d_28_ws.Range(f"C2:C{self.last_row_zm2d_28_ws}").Formula = f'=VLOOKUP(A2,costs,5,FALSE)/VLOOKUP(A2,costs,8,FALSE)'
            self.zm2d_28_file.Close() # close zm2d_28 extract
            # apply filter
            self.new_file_zm2d_28_ws.Range("A1:AD1").AutoFilter(Field=2, Criteria1="YES") 
            # paste screenshots of zm2d_28 first and last page
            try:
                self.new_file_screenshots_ws = self.new_file.Sheets("Screenshots")
                self.pictures = self.new_file_screenshots_ws.Pictures()
                for pic in self.pictures:
                    pic.Delete() # remove all existing screenshots
                    #1
                #C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Cost_consistency/Screenshots/scr2_for_{self.extract_name_zm2d_28[:-5]}.png'
                self.picture_filename = f'scr1_for_{self.extract_name_zm2d_28[:-5]}.png'
                self.picture_path = fr'C:\\Users\\{self.curr_user}\\Desktop\\Automations\\CPT_TP\\Cost_consistency\\Screenshots\\{self.picture_filename}'
                # insert a new screenshot with given parameters
                self.left, self.top, self.width, self.height = 0, 15, 950, 640
                self.picture = self.new_file_screenshots_ws.Shapes.AddPicture(self.picture_path, LinkToFile=False, SaveWithDocument=True, Left=self.left, Top=self.top, Width=self.width, Height=self.height)
                    #2
                self.picture_filename = f'scr2_for_{self.extract_name_zm2d_28[:-5]}.png'
                self.picture_path = fr'C:\\Users\\{self.curr_user}\\Desktop\\Automations\\CPT_TP\\Cost_consistency\\Screenshots\\{self.picture_filename}'
                # insert a new screenshot with given parameters
                self.left, self.top, self.width, self.height = 950, 15, 950, 640
                self.picture = self.new_file_screenshots_ws.Shapes.AddPicture(self.picture_path, LinkToFile=False, SaveWithDocument=True, Left=self.left, Top=self.top, Width=self.width, Height=self.height)
            except Exception as e:
                print(f"No screenshot or {e}")
            self.new_file.Save() # save main Excel file
        except Exception as e:
            print(f"Something happened while copying ZM2D_28, namely {e}")
            self.new_file.Save()
        '''# copy to and from the clipboard -> nice but probably not the best    
            # materials
        # self.new_file_zm2d_28_ws.Range(f"J2:J{self.last_row_zm2d_28_ws}").SpecialCells(12).Copy()
        # win32clipboard.OpenClipboard()
        # self.copied_materials = win32clipboard.GetClipboardData(win32clipboard.CF_UNICODETEXT)
        # win32clipboard.CloseClipboard()
        #     # plants
        # self.new_file_zm2d_28_ws.Range(f"E2:E{self.last_row_zm2d_28_ws}").SpecialCells(12).Copy()
        # win32clipboard.OpenClipboard()
        # self.copied_plants = win32clipboard.GetClipboardData(win32clipboard.CF_UNICODETEXT)
        # win32clipboard.CloseClipboard()'''
    # Here execute second SAP function
        self.run_logic_sap_mb51(self.t_code_sap_mb51) # extract mb51 file from SAP
    
    @excel_decorator
    def run_logic_excel_part_2(self):    
    # reopen main file
    # default Excel runtime optimisation
        # self.excel = win32.Dispatch("Excel.Application")
        # self.excel.AskToUpdateLinks = False
        # self.excel.DisplayAlerts = False
        # self.excel.Visible = True
        # self.excel.ScreenUpdating = False
        self.new_file = self.excel.Workbooks.Open(self.new_file_path) # main file
        self.excel.Application.Calculation = (-4135)  # to set xlCalculationManual # Workbook needs to be opened
        self.new_file_mb51_ws = self.new_file.Worksheets("MB51") # navigate to the MB51 ws
        self.last_row_new_file_mb51_ws = self.new_file_mb51_ws.Cells(1, 1).End(-4121).Row # last row of the current data set
        self.new_file_mb51_ws.Range(f"A2:AA{self.last_row_new_file_mb51_ws}").ClearContents() # prepare the space
    # work with mb51 extract
        self.mb51_file = self.excel.Workbooks.Open(self.mb51_file_path) # open mb51 extract
        self.mb51_ws = self.mb51_file.Sheets(1) # navigate to the extrac
        self.last_row_mb51_ws = self.mb51_ws.Cells(1, 1).End(-4121).Row -1 # we do not need to include the totals
        self.mb51_ws.Range(f"A2:R{self.last_row_mb51_ws}").SpecialCells(12).Copy(Destination=self.new_file_mb51_ws.Range(f"I2")) # copy info
    # add formulas
        self.new_file_mb51_ws.Range(f"A2:A{self.last_row_mb51_ws}").Formula = f'=J2&"-"&K2'
        self.new_file_mb51_ws.Range(f"B2:B{self.last_row_mb51_ws}").Formula = f'=W2/T2'
        self.new_file_mb51_ws.Range(f"C2:C{self.last_row_mb51_ws}").Formula = f'=IF(ISNA(VLOOKUP(A2,costupdates,2,FALSE)),"NO",VLOOKUP(A2,costupdates,2,FALSE))'
        self.new_file_mb51_ws.Range(f"D2:D{self.last_row_mb51_ws}").Formula = f'=IF(C2="NO","-",VLOOKUP(A2,costupdates,3,FALSE))'
        self.new_file_mb51_ws.Range(f"E2:E{self.last_row_mb51_ws}").Formula = f'=IFERROR(D2-B2,"No change")'
        self.new_file_mb51_ws.Range(f"F2:F{self.last_row_mb51_ws}").Formula = f'=IFERROR(E2*T2, "No impact")'
        self.new_file_mb51_ws.Range(f"G2:G{self.last_row_mb51_ws}").Formula = f'=IF(D2="-","NO",IF(ROUND(B2,2)=ROUND(D2,2),"NO","YES"))'
        self.new_file_mb51_ws.Range(f"H2:H{self.last_row_mb51_ws}").Formula = f'=IF(AND(N2="101",O2="GR stock in transit"), "No entry required for movement type 101 GR stock in transit", "")'
        self.new_file_mb51_ws.Range(f"AA2:AA{self.last_row_mb51_ws}").Formula = f"=VLOOKUP(J2,'plant owner'!A:B,2,FALSE)"
    # add filter
        self.new_file_mb51_ws.Range("A1:AA1").AutoFilter(Field=7, Criteria1="YES") 
    # paste screenshots
        try:
            self.new_file_screenshots_ws = self.new_file.Sheets("Screenshots")
            self.pictures = self.new_file_screenshots_ws.Pictures()
                #1
            #C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Cost_consistency/Screenshots/scr2_for_{self.extract_name_zm2d_28[:-5]}.png'
            self.picture_filename = f'scr1_for_mb51_{self.date_stamp}.png'
            self.picture_path = fr'C:\\Users\\{self.curr_user}\\Desktop\\Automations\\CPT_TP\\Cost_consistency\\Screenshots\\{self.picture_filename}'
            # insert a new screenshot with given parameters
            self.left, self.top, self.width, self.height = 0, 640, 950, 640
            self.picture = self.new_file_screenshots_ws.Shapes.AddPicture(self.picture_path, LinkToFile=False, SaveWithDocument=True, Left=self.left, Top=self.top, Width=self.width, Height=self.height)
                #2
            self.picture_filename = f'scr2_for_mb51_{self.date_stamp}.png'
            self.picture_path = fr'C:\\Users\\{self.curr_user}\\Desktop\\Automations\\CPT_TP\\Cost_consistency\\Screenshots\\{self.picture_filename}'
            # insert a new screenshot with given parameters
            self.left, self.top, self.width, self.height = 950, 640, 950, 640
            self.picture = self.new_file_screenshots_ws.Shapes.AddPicture(self.picture_path, LinkToFile=False, SaveWithDocument=True, Left=self.left, Top=self.top, Width=self.width, Height=self.height)
        except Exception as e:
            print(f"No screenshot or {e}")
        self.new_file.Save() # save main Excel file
        # back to default
        self.excel.ScreenUpdating = True
        self.excel.Application.Calculation = -4105  # to set xlCalculationAutomatic
        self.new_file.Close() # close the main file
        self.mb51_file.Close() # close mb51 file
        self.send_email() # send new file by email
 
    @sap_decorator
    def run_logic_sap_mb51(self):
        # self.session is defined inside sap_decorator
        try:
            sap_variant(session=self.session, var_to_use=self.variant_sap_mb51)
        except: 
            sap_variant(session=self.session, var_to_use=self.variant_sap_mb51, version=2) # in our case, V2 is the most probable scenario
        # adjust parameters
        self.session.findById("wnd[0]/usr/btn%_MATNR_%_APP_%-VALU_PUSH").press()
        self.new_file_zm2d_28_ws.Range(f"J2:J{self.last_row_zm2d_28_ws}").SpecialCells(12).Copy()
        self.session.findById("wnd[1]/tbar[0]/btn[24]").press()
        self.session.findById("wnd[1]/tbar[0]/btn[8]").press()
        self.session.findById("wnd[0]/usr/btn%_WERKS_%_APP_%-VALU_PUSH").press()
        self.new_file_zm2d_28_ws.Range(f"E2:E{self.last_row_zm2d_28_ws}").SpecialCells(12).Copy()
        self.session.findById("wnd[1]/tbar[0]/btn[24]").press()
        self.session.findById("wnd[1]/tbar[0]/btn[8]").press()
        self.session.findById("wnd[0]/usr/ctxtBUDAT-LOW").text = self.last_month_first_day
        self.session.findById("wnd[0]/usr/ctxtBUDAT-HIGH").text = self.last_month_last_day
        self.session.findById("wnd[0]/usr/ctxtALV_DEF").text = self.layout_sap_mb51
        # run the report
        sap_run(session=self.session)
        sap_screen_nagivation(session=self.session, action="up")
        #self.session.findById("wnd[0]/usr/lbl[1,1]").setFocus() # ensure report is loaded
            #scr1
        try: # first page
            screenshot_first_page_mb51 = pyautogui.screenshot()
            screenshot_first_page_mb51.save(fr'C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Cost_consistency/Screenshots/scr1_for_mb51_{self.date_stamp}.png')
        except Exception as e: print(e)
        sap_screen_nagivation(session=self.session, action="down")
        time.sleep(2)
            #scr2
        try: # last page
            screenshot_last_page_mb51 = pyautogui.screenshot()
            screenshot_last_page_mb51.save(fr'C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/Cost_consistency/Screenshots/scr2_for_mb51_{self.date_stamp}.png')
        except Exception as e: print(e)
        self.session.findById("wnd[0]/tbar[1]/btn[48]").press() # change the layout
        sap_extract(session=self.session, extr_path=self.extract_path_mb51, extr_name=self.extract_name_mb51) #extract the report
        close_excel()

control = Control()
control()
#control.run_logic_sap_fagll03(control.t_code_sap)
#control.run_logic_sap_faglb03(control.t_code_sap_faglb03)

113064


In [ ]:
import numpy as np
import pandas as pd

#--GIT APxx-xx tab:
# looks like a very nasty .txt file, which requires specific approach
df_sap = pd.read_csv("C:/Users/KOMAROVO/Desktop/Python_Main/Python_Work/GIT/GIT_reference_files/GIT_AP02.txt", 
                     on_bad_lines='skip', sep="\t", encoding="ANSI",skiprows=13, skipinitialspace = True)
df_sap = df_sap.loc[:, ~df_sap.columns.str.contains('^Unnamed')] # drop all unnamed columns -> ~ stands for bool
df_sap = df_sap[df_sap['Plant'] != "Plant"] # drop all rows that have "Plant" in their names (those are repetitions of headers)
df_sap.dropna(subset=['Plant'], inplace=True)
# convert respective columns to numeric
df_sap['Quantity'] = df_sap['Quantity'].str.replace(',', '').astype(float)
df_sap['Amount in LC'] = df_sap['Amount in LC'].str.replace(',', '').astype(float)
df_sap['Net Order Value in PO Curr.'] = df_sap['Net Order Value in PO Curr.'].str.replace(',', '').astype(float)
df_sap['PO Quantity'] = df_sap['PO Quantity'].str.replace(',', '').astype(float)
# export to excel
#df_sap.to_excel("C:/Users/KOMAROVO/Desktop/Python_Main/Python_Work/GIT/GIT_reference_files/Pandas_GIT_AP02.xlsx", index=False)

#--Data Reworked APxx-xx tab:
df_sap_le = df_sap.copy() # creating a copy of the file with only 4 LEs 
df_sap_le = df_sap_le[(df_sap_le['Company Code'] == "2941") | (df_sap_le['Company Code'] == "2942") | 
                      (df_sap_le['Company Code'] == "2946") | (df_sap_le['Company Code'] == "2951")]
# reset index:
df_sap_le = df_sap_le.reset_index(drop=True)
# add index column:
df_sap_le['INDEX'] = df_sap_le.index + 2
# adding columns:
# concatenate:
df_sap_le.insert(loc=0, column='Concatenate', value=df_sap_le['Material Number'] + "-" + df_sap_le['Plant'])
# other:
df_sap_le['BUoM historic'] = "=VLOOKUP(A" + df_sap_le['INDEX'].astype(str) + ",historicalcostsv2,7,FALSE)"
df_sap_le['Std price per BUoM'] = "=VLOOKUP(A" + df_sap_le['INDEX'].astype(str) + ",historicalcostsv2,9,FALSE)"
df_sap_le['Value at historical cost'] = "=IF(AJ" + df_sap_le['INDEX'].astype(str) + "=AC" + df_sap_le['INDEX'].astype(str) + ",0,D" + df_sap_le['INDEX'].astype(str) + "*AO" + + df_sap_le['INDEX'].astype(str) + ")"
df_sap_le['Test UOM'] = "=AN" + df_sap_le['INDEX'].astype(str) + "=E" + df_sap_le['INDEX'].astype(str)
df_sap_le['Diff $'] = "=IF(AJ" + df_sap_le['INDEX'].astype(str) + "=AC" + df_sap_le['INDEX'].astype(str) + ",0,AP" + df_sap_le['INDEX'].astype(str) + "-F" + df_sap_le['INDEX'].astype(str) + ")"
print(df_sap_le)
# drop index colums:
df_sap_le = df_sap_le.drop('INDEX', axis=1)
# export to excel:
#df_sap_le.to_excel("C:/Users/KOMAROVO/Desktop/Python_Main/Python_Work/GIT/GIT_reference_files/Pandas_GIT_Reworked.xlsx", index=False)


#print(df_sap_le)


# second part (work with mm60 extract)

In [ ]:
import numpy as np
import pandas as pd

df_mm60 = pd.read_excel("C:/Users/KOMAROVO/Desktop/Python_Main/Python_Work/GIT/GIT_reference_files/Historical cost AP02 LE2941.xlsm", 
                        sheet_name="MM60 Report", converters={'Price':float})

pd.options.display.float_format = '{:20,.2f}'.format # handling scientific notation

# add columns:
#df_mm60['INDEX'] = df_mm60.index + 2
df_mm60.insert(loc=0, column='Concatenate', value=df_mm60['Material'] + "-" + df_mm60['Plant'])
df_mm60['Std price per unit'] = df_mm60['Price'] / df_mm60['Price unit']
print(df_mm60)
df_mm60.to_excel("C:/Users/KOMAROVO/Desktop/Python_Main/Python_Work/GIT/GIT_reference_files/pandas_mm60_rpa.xlsx", index=False)

# if historical cost file is received from Roel

In [84]:
import numpy as np
import pandas as pd

df_roel = pd.read_excel("C:/Users/KOMAROVO/Desktop/Python_Main/Python_Work/GIT/GIT_reference_files/historical costs AP 2 2023 - LE 2941.xlsx",
                        converters={'BusA': str})
# remove all unwanted columns:
df_roel = df_roel[['Plnt', 'Material', 'BusA', 'Standard price', 'Crcy', 'BUn', 'per']]
# add useful columns:
df_roel.insert(loc=0, column='Concatenate', value=df_roel['Material'] + "-" + df_roel['Plnt'])
df_roel['Std price per unit'] = df_roel['Standard price'] / df_roel['per']
# drop last row:
df_roel.drop(df_roel.tail(1).index,inplace=True) # drop last (n) rows

print(df_roel)

          Concatenate  Plnt  Material  BusA       Standard price  Crcy  BUn  \
0       10009694-AT00  AT00  10009694  9334                 6.33   NaN  SYR   
1       10020218-AT00  AT00  10020218  9334            19,562.83   NaN   PC   
2       10020219-AT00  AT00  10020219  9334               861.93   NaN   PC   
3       10020220-AT00  AT00  10020220  9334            17,135.68   NaN   PC   
4       10020221-AT00  AT00  10020221  9334             5,327.76   NaN   PC   
...               ...   ...       ...   ...                  ...   ...  ...   
107048  25001338-IE11  IE11  25001338  7246                 5.67   NaN   SU   
107049  25001339-IE11  IE11  25001339  7246                12.05   NaN   SU   
107050  25001340-IE11  IE11  25001340  7246                35.70   NaN   SU   
107051  25001341-IE11  IE11  25001341  7246                54.44   NaN   SU   
107052  25001342-IE11  IE11  25001342  7246                33.00   NaN   SU   

                        per   Std price per unit  
